# Model zainspirowany filmem znalezionym na yt:https://youtu.be/doDUihpj6ro, zmieniony jedynie na potrzeby mojego projektu.

### Import modułów używanych do projektu:

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [4]:
mp_holistic = mp.solutions.holistic          #import modelu wykrywającego cialo mediapipe
mp_drawing = mp.solutions.drawing_utils      #import narzędzi do oznaczania detekcji na obrazie

### Funkcja pozwalająca przeprocesować klatki filmu modelowi mediapipe:

In [5]:
def mediapipe_det(image,model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #konwersja kolorów BGR na RGB klatki
    image.flags.writeable = False                  #zarządzanie pamięcią dla np.array
    results = model.process(image)                 #processowanie klatki za pomocą modelu mediapipe
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

### Funkcja zwracająca klatki z narysowanymi liniamii dłoni:

In [6]:
def draw_hands(image, results):
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

### Funkcja otwierająca oraz zapisująca wideo, oraz implementująca poprzednie funkcje napisane wyżej:

In [29]:
cap = cv2.VideoCapture('trim1.mp4')                            #otwarcie pliku ze ścieżki
size = (int(cap.get(3)), int(cap.get(4)))                      #określenie rozdzielczości
fps =  int(cap.get(cv2.CAP_PROP_FPS))                          #określenie fps filmu
output = cv2.VideoWriter('test1.mp4', cv2.VideoWriter.fourcc(*'mp4v'), fps,size)              #funkcja opencv zapisująca wynik

with mp_holistic.Holistic(min_detection_confidence=0.3, min_tracking_confidence = 0.3) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()  #odczytanie klatek z filmu, ret - ilość pozostałych klatek, frame - "obraz" klatki
        if not ret:                                                 # przerwanie pętli, kiedy nie ma juz pozostałych klatek do przerobienia
            break
            
        image, results = mediapipe_det(frame, holistic)             # procesowanie klatki przez model mediapipe
        
        draw_hands(image, results)                                  # rysowanie lini dłoni
            
        output.write(image)                                         # zapisywanie zmienionych klatek i łączenie je z powrotem w film

        cv2.imshow("OpenCV Feed", image)                            # podgląd "live"
        if cv2.waitKey(0) & 0xFF == ord('q'):                       # zamknięcie okna opencv za pomoca przycisku 'q'
            break

    cap.release()
    output.release()
    cv2.destroyAllWindows()

success
